# Estimate market factors of financial time series (FTS)

In [1]:
import re
import sys
import warnings
import pandas as pd  # type: ignore

sys.path.append("../modules")
import estimate_market_factors as emf  # type: ignore  # noqa: E402

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
# pd.options.display.max_rows = 277

## Global variables

In [2]:
input_path_raw = "../input_files/raw_data"
input_path_processed = "../input_files/processed_data"
input_path_data_dictionary = "../input_files/data_dictionary"
log_path = "../logs"
output_path = "../output_files"
input_generation_date = "2024-08-31"

time_delta = 250
time_step = 20

## Load data and filter duplicates by symbol and date

In [3]:
# Load data and filter duplicates by symbol and date
df_stock_index = pd.read_csv(
    "{}/df_all_markets_{}.csv".format(
        input_path_processed, re.sub("-", "", input_generation_date)
    ),
    low_memory=False,
)  # noqa: E501

df_stock_index.drop_duplicates(
    subset=["market_ticker", "date", "symbol"],
    keep="first",
    inplace=True,
    ignore_index=True,
)
df_stock_index["date"] = pd.to_datetime(
    df_stock_index["date"], errors="coerce"
)  # noqa: E501

In [4]:
df_stock_index["min_date"] = df_stock_index.groupby(["market_ticker"])[
    "date"
].transform(
    "min"
)  # noqa: E501
df_stock_index["max_date"] = df_stock_index.groupby(["market_ticker"])[
    "date"
].transform(
    "max"
)  # noqa: E501
df_stock_index[
    ["market_ticker", "market_name", "market_country", "min_date", "max_date"]
].value_counts(
    dropna=False, sort=False
).reset_index()  # noqa: E501

,market_ticker,market_name,market_country,min_date,max_date,count
0,GD.AT,Athens Stock Exchange,Greece,2000-01-04,2024-08-30,261845
1,ICOLCAP.CL,iShares MSCI COLCAP Stock Fund,Colombia,2000-01-04,2024-08-30,82547
2,^BUX.BD,Budapest Stock Index,Hungary,2000-01-05,2024-08-30,52836
3,^CASE30,Cairo and Alexandria Stock Exchange,Egypt,2000-11-30,2024-08-29,84772
4,^DJI,Dow Jones Industrial Average,United States,2000-01-04,2024-08-30,176972
5,^FCHI,Continuous Assisted Trading 40,France,2000-01-04,2024-08-30,238755
6,^GDAXI,DAX Performance Index,Germany,2000-01-04,2024-08-30,219509
7,^GSPC,Standard and Poor's 500 - S&P 500,United States,2000-01-04,2024-08-30,2731228
8,^IBEX,Iberian Index 35,Spain,2000-01-04,2024-08-30,170134
9,^IPSA,Chilean stock market index,Chile,2000-01-04,2024-08-30,131724


## Rolling window analysis

In [5]:
def get_market_list(df_stock_index, time_delta, time_step):
    market_args_list = (
        df_stock_index[["date"]]
        .sort_values(by=["date"])
        .value_counts(sort=False)
        .reset_index()
    )
    market_args_list["final_date"] = market_args_list["date"] + pd.DateOffset(
        days=time_delta
    )
    market_args_list = (
        market_args_list.rename(columns={"date": "initial_date"})
        .iloc[::time_step]
        .reset_index()
        .drop(columns=["count", "index"])
    )
    market_args_list["initial_date"] = market_args_list["initial_date"].astype(str)  # noqa: E501
    market_args_list["final_date"] = market_args_list["final_date"].astype(str)
    market_args_list  # .values.tolist()

    return market_args_list

In [6]:
print(df_stock_index["market_ticker"].unique())
df_stock_index = df_stock_index[df_stock_index["market_ticker"] == "^FCHI"]

['^BUX.BD' '^CASE30' '^DJI' '^FCHI' 'GD.AT' '^GDAXI' '^GSPC' '^IBEX'
 'ICOLCAP.CL' '^IPSA' '^JKSE' '^MXX' '^NSEI']


In [7]:
# time_step = 480

for market_ticker in df_stock_index["market_ticker"].unique():
    # Local parameters per market
    stock_index_name = re.sub("(\^)|(=X)", "", market_ticker.lower())  # noqa: W605
    market_name = df_stock_index[df_stock_index["market_ticker"] == market_ticker]["market_name"].unique()[0]  # noqa: E501
    market_country = df_stock_index[df_stock_index["market_ticker"] == market_ticker]["market_country"].unique()[0]  # noqa: E501
    market_type = df_stock_index[df_stock_index["market_ticker"] == market_ticker]["market_type"].unique()[0]  # noqa: E501
    print("----------------------------------- {} -----------------------------------\n".format(market_ticker))  # noqa: E501
    print("- Filename: {}\n- Name: {}\n- Country: {}\n- Type: {}\n".format(stock_index_name, market_name, market_country, market_type))  # noqa: E501

    # Loop of dates per market
    market_args_list = get_market_list(
        df_stock_index=df_stock_index[df_stock_index["market_ticker"] == market_ticker],  # noqa: E501
        time_delta=time_delta,
        time_step=time_step,
    )

    # Estimate Market Efficiency
    df_normalized, df_normalized_eigenvalues = [], []
    df_residuals, df_residuals_eigenvalues = [], []

    for k in range(market_args_list.shape[0]):
        initial_date = market_args_list["initial_date"].values[k]
        final_date = market_args_list["final_date"].values[k]

        # Normalized Returns
        try:
            df_aux = emf.get_market_efficiency(
                df=df_stock_index[df_stock_index["market_ticker"] == market_ticker],  # noqa: E501
                column_="z_score_log_return",
                min_bins=10,
                precision=12,
                log_path=log_path,
                log_filename="log_rolling_window_{}".format(stock_index_name),
                log_filename_entropy="log_entropy_rolling_{}".format(stock_index_name),  # noqa: E501
                verbose=1,
                tqdm_bar=True,
                market_args_list=(initial_date, final_date),
                bouchaud_filter=False,
                n=int(df_stock_index.shape[0] * 2),
                df_tracy_widom=pd.read_csv(
                    "{}/tracy_widom.csv".format(input_path_data_dictionary),
                    low_memory=False,
                ),
                alphas=[0.01, 0.05, 0.10],
                k_max=8,
                df_onatski=pd.read_csv(
                    "{}/onatski.csv".format(input_path_data_dictionary),
                    low_memory=False,
                ),
                levels=[1, 2, 5, 10],
            )

            df_normalized.append(df_aux[0])
            df_normalized_eigenvalues.append(df_aux[1])

            print("Finished: {} - {}".format(initial_date, final_date))
        except Exception:
            print("No done: {} - {}".format(initial_date, final_date))

        # Normalized residuals
        try:
            df_aux = emf.get_market_efficiency(
                df=df_stock_index[df_stock_index["market_ticker"] == market_ticker],  # noqa: E501
                column_="z_score_zlr_no_market",
                min_bins=10,
                precision=12,
                log_path=log_path,
                log_filename="log_rolling_window_residuals_{}".format(stock_index_name),  # noqa: E501
                log_filename_entropy="log_entropy_rolling_residuals_{}".format(
                    stock_index_name
                ),
                verbose=1,
                tqdm_bar=True,
                market_args_list=(initial_date, final_date),
                bouchaud_filter=False,
                n=int(df_stock_index.shape[0] * 2),
                df_tracy_widom=pd.read_csv(
                    "{}/tracy_widom.csv".format(input_path_data_dictionary),
                    low_memory=False,
                ),  # noqa: E501
                alphas=[0.01, 0.05, 0.10],
                k_max=8,
                df_onatski=pd.read_csv(
                    "{}/onatski.csv".format(input_path_data_dictionary),
                    low_memory=False,
                ),  # noqa: E501
                levels=[1, 2, 5, 10],
            )

            df_residuals.append(df_aux[0])
            df_residuals_eigenvalues.append(df_aux[1])

            print("Finished: {} - {}".format(initial_date, final_date))
        except Exception:
            print("No done: {} - {}".format(initial_date, final_date))

    # Merge final dataframe per market
    df_normalized = pd.concat(df_normalized)
    df_normalized_eigenvalues = pd.concat(df_normalized_eigenvalues)

    df_residuals = pd.concat(df_residuals)
    df_residuals_eigenvalues = pd.concat(df_residuals_eigenvalues)

    df_normalized["market_ticker"] = market_ticker
    df_normalized["market_name"] = market_name
    df_normalized["market_country"] = market_country
    df_normalized["market_type"] = market_type

    df_normalized_eigenvalues["market_ticker"] = market_ticker
    df_normalized_eigenvalues["market_name"] = market_name
    df_normalized_eigenvalues["market_country"] = market_country
    df_normalized_eigenvalues["market_type"] = market_type

    df_residuals["market_ticker"] = market_ticker
    df_residuals["market_name"] = market_name
    df_residuals["market_country"] = market_country
    df_residuals["market_type"] = market_type

    df_residuals_eigenvalues["market_ticker"] = market_ticker
    df_residuals_eigenvalues["market_name"] = market_name
    df_residuals_eigenvalues["market_country"] = market_country
    df_residuals_eigenvalues["market_type"] = market_type

    # Local Saving of files
    temp = "{}/df_{}".format(output_path, stock_index_name)
    filename = "{}_rolling_window_{}.csv".format(
        temp, re.sub("-", "", input_generation_date)
    )
    filename_e = "{}_eigenvalues_evolution_{}.csv".format(
        temp, re.sub("-", "", input_generation_date)
    )
    filename_r = "{}_rolling_window_residuals_{}.csv".format(
        temp, re.sub("-", "", input_generation_date)
    )
    filename_re = "{}_eigenvalues_evolution_residuals_{}.csv".format(
        temp, re.sub("-", "", input_generation_date)
    )  # noqa: E501

    df_normalized.to_csv(filename, index=False)
    df_residuals.to_csv(filename_r, index=False)
    df_normalized_eigenvalues.to_csv(filename_e, index=False)
    df_residuals_eigenvalues.to_csv(filename_re, index=False)

    print("----------------------------------- DONE - {} -----------------------------------\n".format(market_ticker))  # noqa: E501

----------------------------------- ^FCHI -----------------------------------

- Filename: fchi
- Name: Continuous Assisted Trading 40
- Country: France
- Type: Developed



100%|████████████████████| 465/465 [00:00<00:00, 524.35it/s]


Finished: 2000-01-04 - 2000-09-10


100%|████████████████████| 465/465 [00:01<00:00, 435.41it/s]


Finished: 2000-01-04 - 2000-09-10


...

No done: 2024-07-30 - 2025-04-06


0it [00:00, ?it/s]


No done: 2024-08-27 - 2025-05-04


0it [00:00, ?it/s]


No done: 2024-08-27 - 2025-05-04
----------------------------------- DONE - ^FCHI -----------------------------------

